In [1]:
from nltk.corpus import words
import nltk
# nltk.download("words")
import re
import pandas as pd
import numpy as np

In [2]:
# copy pasted from wikipedia
# right side words are most commonly mispelled

mis_spelled_words="""absence – absense, absentse, abcense, absance[3][10]
acceptable – acceptible[4]
accidentally/accidently – accidentaly[4]
accommodate – accomodate, acommodate[3][4]
achieve – acheive[3]
acknowledge – acknowlege, aknowledge[3]
acquaintance – acquaintence, aquaintance[3]
acquire – aquire, adquire[4]
acquit – aquit[4]
calendar – calender[3][4]
camouflage – camoflage, camoflague[3]
capitol – capital[3] (both words exist, but are distinct)
Caribbean – Carribean[3]
category – catagory[3][4]
caught – cauhgt, caugt[3]
cemetery – cemetary,[1] cematery[3]
changeable – changable[3][4]
chief – cheif[3]
embarrass – embarass[1][4]
equipment – equiptment (wrong in numerous webpages)[4]
exceed – excede[4]
exhilarate – exilerate[4]
ignorance – ignorence[4]
imitate – immitate[3]
immediately – imediately[
readable – readible[8]
really – realy[1]
receive – recieve[1][4]
receipt – reciept[4]
tyranny – tyrany[4]
underrate – underate[4]
until – untill[4]
upholstery – upholstry[4]
welfare – wellfare, welfair[3]
whether – wether[3]
wilful – wilfull (American: willful)[1]
withhold – withold[1]
writing – writting, writeing[10]
skilful – skilfull (American: skillful)[1]
speech – speach, speeche (archaic)[10]
successful – succesful, successfull, sucessful[1]
supersede – supercede[4]
surprise – suprise, surprize[1]
playwright – playright, playwrite[4]
possession – posession, possesion[1][4]
potatoes – potatos[1]
precede – preceed[4]
presence – presance[1]
principle – principal[4]
height – heighth, heigth[4]
hierarchy – heirarchy[4]
hors d'oeuvres – hors derves, ordeurves[3]
humorous – humerous[4]
hygiene – hygene, hygine, hiygeine, higeine, hygeine[3]
hypocrisy/hypocrite – hipocrit[1]"""

In [3]:
mis_spelled_words=mis_spelled_words.split("\n")  #making doc string to list of sentence

mis_spelled_words=[i.split("–")[1].strip()  for i in mis_spelled_words]   #splitting them and get only wrong words

list_of_words=[]          # storing words in array
for i in mis_spelled_words:
    list_of_words.append(re.sub("[^a-zA-Z]"," ",i).split()[0])         # getting only words 

In [4]:
df=pd.DataFrame({"misspelled_words":list_of_words})

In [7]:
print(df.shape)
df.head(5)

(55, 1)


,misspelled_words
0,absense
1,acceptible
2,accidentaly
3,accomodate
4,acheive


# EDIT DISTANCE/LEVENSHTEIN DISTANCE

In [8]:
# USING EDIT DISTANCE METHOD TO FIND SIMILAR WORDS 
# HAVING MINIMUM DISTANCE IS TAKEN

In [9]:
def relation(w1,w2,show_arr=True):
    w1="#"+w1
    w2="#"+w2
    arr=[[0]*len(w1)]*len(w2)
    arr=np.zeros(len(w1)*len(w2)).reshape((len(w2),len(w1)))
    arr[0,:]=[*range(0,len(w1))]
    arr[:,0]=[*range(0,len(w2))]

    for i in range(1,len(w2)):
        for j in range(1,len(w1)):
            if w1[j]==w2[i]:
    #             print(i,j)
                arr[i,j]=arr[i-1,j-1]
    #             print(w1[j],w2[i],"==>",arr[i,j])
            else:
    #             print(i,j)
                arr[i,j]=min(arr[i-1,j],arr[i,j-1],arr[i-1,j-1])+1
#     print("The distance between them ",arr[-1,-1])
    if show_arr:
        return arr[-1,-1],w2[1:]

In [10]:
# EG:
relation("chennai","channae")  # returns diatance and word

(2.0, 'channae')

In [11]:
word_dict=words.words()                   # nltk getting list of all words
print("the number of words the nltk has :",len(word_dict))
def nearest_words(word):
    
    same_first_letter=[]
    similar_words=[]
    for i in word_dict:
        if i.startswith(word[0]):                    #assuming first letter is same for correct and misspelled words
            same_first_letter.append(i)                 # to filter the words it is used
            
    for i in same_first_letter:
        similar_words.append(relation(word,i))                            # getting distance for each word
    similar_words=sorted(similar_words,key=lambda x:x[0],reverse=False)   # sorting based on distance
    return similar_words

the number of words the nltk has : 236736


In [12]:
# EG:
nearest_words("avid")[:10]

[(0.0, 'avid'),
 (1.0, 'acid'),
 (1.0, 'aid'),
 (1.0, 'amid'),
 (1.0, 'arid'),
 (1.0, 'avoid'),
 (1.0, 'acid'),
 (2.0, 'abed'),
 (2.0, 'abide'),
 (2.0, 'abidi')]

In [13]:
df["correct_words1"]=df["misspelled_words"].apply(lambda x :nearest_words(x)[0][1])
df["correct_words2"]=df["misspelled_words"].apply(lambda x :nearest_words(x)[1][1])

In [14]:
df.head(10)

,misspelled_words,correct_words1,correct_words2
0,absense,absence,abscise
1,acceptible,acceptable,accendible
2,accidentaly,accidental,accidentally
3,accomodate,accommodate,accommodable
4,acheive,achene,achete
5,acknowlege,acknowledge,acknowledged
6,acquaintence,acquaintance,acquaintancy
7,aquire,acquire,acquired
8,aquit,acquit,absit
9,calender,calender,calander
